## AutoGen

安装主要的依赖包
```
pip install pyautogen openai azure-identity yfinance
```

In [1]:
import openai
from azure.identity import ClientSecretCredential

client_id = "################"
client_secret = "#################"
tenant_id = "#################"
endpoint = "###############"

# Request credential
sp_credential = ClientSecretCredential(
    client_id=client_id,
    client_secret=client_secret,
    tenant_id=tenant_id)
token = sp_credential.get_token("https://cognitiveservices.azure.com/.default")
 
# use aad token
openai.api_type = "azure_ad"
openai.api_key = token.token
openai.api_base = endpoint
openai.api_version = "2023-12-01-preview"

- 配置模型，在此使用 gpt-4-turbo
- 定义不同 agent 的任务

In [2]:
import autogen

config_list = [{"model": "gpt-4-turbo", 
                "api_key": openai.api_key,
                "base_url":endpoint,
                "api_type": 'azure_ad',
                "api_version": openai.api_version}]
llm_config = {"config_list": config_list}

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

构造 multi-agent 系统
- financial_assistant，主要负责 financial_tasks[0] 任务
- research_assistant，主要负责 financial_tasks[1] 任务
- writer，主要负责 writing_tasks 任务
- user，主要的提问者

然后按照任务顺序开始对话。

In [3]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "last_msg",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure or a table of data in the blogpost.",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To fetch the current stock prices of NVIDIA (NVDA) and Tesla (TSLA), along with the performance over the past month in terms of percentage change, we are going to use the `yfinance` library in Python.

First, you'll need to install the `yfinance` package if you haven't already. You can install it using `pip`. Then, we will write the Python code to

输出每个任务的最终输出结果。

In [4]:
for i, chat_res in enumerate(chat_results):
    print(f"*****{i}th chat*******:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

*****0th chat*******:
The execution of the code has successfully fetched the required stock data:

- NVIDIA (NVDA) has a current stock price of $726.13, with a 1-month percentage change of +21.72%.
- Tesla (TSLA) has a current stock price of $199.95, with a 1-month percentage change of -4.24%.

These figures represent their most recent closing prices and the performance over the past month relative to their prices 30 days ago.


Human input in the middle: []
Conversation cost:  ({'total_cost': 0.08499, 'gpt-4': {'cost': 0.08499, 'prompt_tokens': 1591, 'completion_tokens': 621, 'total_tokens': 2212}}, {'total_cost': 0})



*****1th chat*******:
The stock performance for NVIDIA showed a significant increase of +21.72% over the past month, whereas Tesla experienced a decrease of -4.24% in the same time frame. Possible reasons for this might include company earnings reports, analyst ratings, industry trends, market conditions, news and events impacting the companies, technological advancem